### Cele doua norme
- Norma 1: ||A||_1
- Norma infinit: ||A||_inf

In [1]:
import numpy as np

In [3]:
def norma_1(A):
    return np.max(np.sum(np.abs(A), axis=0)) # col

def norma_infinit(A):
    return np.max(np.sum(np.abs(A), axis=1)) # linie

In [4]:
A = np.array([
    [1,  2,  3],
    [-4, 5, -6]
])

print(f'norma 1: {norma_1(A)}') 
print(f'norma infinit: {norma_infinit(A)}')

norma 1: 9
norma infinit: 15


### Alegerea matricei initiale V0 
- Folosind (5) si (6)

In [5]:
def alegere_V0(A):
    norma_A_1 = norma_1(A)
    norma_A_inf = norma_infinit(A)
    AT = A.T
    V0 = AT / (norma_A_1 * norma_A_inf)
    return V0

In [6]:
alegere_V0(A)

array([[ 0.00740741, -0.02962963],
       [ 0.01481481,  0.03703704],
       [ 0.02222222, -0.04444444]])

### Metode iterative de aproximare a inversei unei matrice

1. Metoda Schultz

In [7]:
def metoda_schultz(A, n, epsilon, kmax):
    V0 = alegere_V0(A)
    V1 = V0.copy()
    k = 0
    I = np.eye(n)
    
    while k < kmax:
        V0 = V1.copy()
        
        AV0 = A @ V0 # @ in loc de np.matmul()
        V1 = V0 @ (2 * I - AV0)
        
        diff = np.linalg.norm(V1 - V0, 1)
        
        if diff < epsilon:
            return V1, k, True
        
        if diff > 1e10:
            return V1, k, False
        
        k += 1
    
    return V1, k, False

In [9]:
A = np.array([
    [1, 2, 0, 0],
    [0, 1, 2, 0],
    [0, 0, 1, 2],
    [0, 0, 0, 1]
])

metoda_schultz(A, 4, 1e-10, 10000)

(array([[ 1.00000000e+00, -2.00000000e+00,  4.00000000e+00,
         -8.00000000e+00],
        [ 1.45434799e-29,  1.00000000e+00, -2.00000000e+00,
          4.00000000e+00],
        [-6.68144552e-30,  1.77156230e-29,  1.00000000e+00,
         -2.00000000e+00],
        [ 2.70202537e-30, -7.04140569e-30,  1.41875768e-29,
          1.00000000e+00]]),
 15,
 True)